In [11]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
breastcancerdata = load_breast_cancer()
X = breastcancerdata.data
y = breastcancerdata.target
# 创建样本数据
'''X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=0, random_state=7)'''
y = 2*y-1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

def stump_classify(X, dimension, thresh_val, thresh_ineq):
    """通过阈值比较对数据进行分类的决策树桩分类器"""
    ret_array = np.ones((np.shape(X)[0], 1))
    if thresh_ineq == 'lt':
        ret_array[X[:, dimension] <= thresh_val] = -1.0
    else:
        ret_array[X[:, dimension] > thresh_val] = -1.0
    return ret_array

def build_stump(X, y, D):
    """构建最佳单层决策树"""
    m, n = np.shape(X)
    num_steps = 10.0
    best_stump = {}
    best_class_est = np.zeros((m, 1))
    min_error = np.inf
    for i in range(n):
        range_min = X[:, i].min()
        range_max = X[:, i].max()
        step_size = (range_max - range_min) / num_steps
        for j in range(-1, int(num_steps) + 1):
            for inequal in ['lt', 'gt']:
                thresh_val = (range_min + float(j) * step_size)
                predicted_vals = stump_classify(X, i, thresh_val, inequal)
                err_arr = np.ones((m, 1))
                err_arr[predicted_vals == y] = 0
                weighted_error = np.dot(D.T, err_arr)
                if weighted_error < min_error:
                    min_error = weighted_error
                    best_class_est = predicted_vals.copy()
                    best_stump['dim'] = i
                    best_stump['thresh'] = thresh_val
                    best_stump['ineq'] = inequal
    return best_stump, min_error, best_class_est

class AdaBoostClassifier_(BaseEstimator, ClassifierMixin):
    def __init__(self, num_iters=40):
        self.num_iters = num_iters
        self.classifiers = []
        
    def fit(self, X, y):
        y = y.reshape(-1, 1)  # 确保 y 是列向量
        m = X.shape[0]
        D = np.ones((m, 1)) / m
        agg_class_est = np.zeros((m, 1))
        for i in range(self.num_iters):
            '''code here'''
            best_stump['alpha'] = alpha
            self.classifiers.append(best_stump)
            '''code here'''
            agg_class_est += alpha * class_est
            agg_errors = np.multiply(np.sign(agg_class_est) != y, np.ones((m, 1)))
            error_rate = agg_errors.sum() / m
            if error_rate == 0.0:
                break
        return self
    
    def predict(self, X):
        m = X.shape[0]
        agg_class_est = np.zeros((m, 1))
        for classifier in self.classifiers:
            class_est = stump_classify(X, classifier['dim'], classifier['thresh'], classifier['ineq'])
            agg_class_est += classifier['alpha'] * class_est
        return np.sign(agg_class_est)

   



In [13]:
from sklearn.ensemble import AdaBoostClassifier
# 创建模型
clf1 = AdaBoostClassifier_()
clf2 = AdaBoostClassifier()
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
# 进行预测
y_pred1 = clf1.predict(X_test)
# 评估模型
print("Accuracy of clf1:", accuracy_score(y_test, y_pred1))
print("Accuracy of clf2:", clf2.score(X_test, y_test))

C:\Users\25751\AppData\Local\Temp\ipykernel_20896\1384536587.py:51: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  alpha = float(0.5 * np.log((1.0 - error) / max(error, 1e-16)))


Accuracy of clf1: 0.9649122807017544
Accuracy of clf2: 0.9766081871345029
